In [1]:
import os

In [2]:
pwd

'/Users/aksshar/PycharmProjects/Knee-Osteoarthritis-Severity-Grading/research'

In [9]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config["artifacts_root"]])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config['data_ingestion']

        create_directories([config['root_dir']])

        data_ingestion_config= DataIngestionConfig(
            root_dir= config['root_dir'],
            source_URL= config['source_URL'],
            local_data_file= config['local_data_file'],
            unzip_dir= config['unzip_dir']
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-16 00:36:59,119: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-16 00:36:59,121: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-16 00:36:59,122: INFO: common: created directory at: artifacts]
[2023-07-16 00:36:59,123: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-16 00:37:03,340: INFO: 324913937: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23046483
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7cb504e94582cbba0b35ee3f19e40c505650d352be2adc892683f9677eaa3ad7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CF92:1322E:28EC6F8:2A8D0A9:64B32D9B
Accept-Ranges: bytes
Date: Sat, 15 Jul 2023 23:37:00 GMT
Via: 1.1 varnish
X-Served-By: cache-lhr7352-LHR
X-Cache: M